# DCOPF

In [1]:
import scipy
import sympy as smp
import numpy as np
import pandas as pd

import andes
from andes.shared import rad2deg
import ams

from ams.solver.pypower.makePTDF import makePTDF
from ams.solver.pypower.makeBdc import makeBdc

import pandapower as pp

from pandapower.pypower.makePTDF import makePTDF as ppmakePTDF
from pandapower.pd2ppc import _pd2ppc

import pandapower.networks as ppnet
from scipy.sparse import lil_matrix, csr_matrix, eye, linalg


In [2]:
andes.config_logger(stream_level=10)

In [3]:
sa = andes.load(andes.get_case('ieee14/ieee14_ace.xlsx'),
              setup=True,)

Working directory: "/Users/jinningwang/Documents/work/ams/dev/demo"
Found files: ['/Users/jinningwang/Documents/work/mambaforge/envs/ams/lib/python3.9/site-packages/andes/cases/ieee14/ieee14_ace.xlsx']
> Loaded config from file "/Users/jinningwang/.andes/andes.rc"
> Loaded generated Python code in "/Users/jinningwang/.andes/pycode".
Input format guessed as xlsx.
Parsing input file "/Users/jinningwang/Documents/work/mambaforge/envs/ams/lib/python3.9/site-packages/andes/cases/ieee14/ieee14_ace.xlsx"...
Input file parsed in 0.0427 seconds.
Setting internal address for Bus
Setting internal address for PQ
Setting internal address for PV
Setting internal address for Slack
Setting internal address for Shunt
Setting internal address for Line
Setting internal address for Area
System internal structure set up in 0.0267 seconds.


In [4]:
ams.config_logger(stream_level=10)

In [5]:
sp = ams.load(ams.get_case('matpower/case14.m'),
              setup=True,)

ppc = ams.io.pypower.system2ppc(sp)

ppn = ppnet.case14()
pp.rundcpp(ppn)
_, ppci = _pd2ppc(ppn)

Working directory: "/Users/jinningwang/Documents/work/ams/dev/demo"
Found files: ['/Users/jinningwang/Documents/work/ams/ams/cases/matpower/case14.m']
Input format guessed as matpower.
Parsing input file "/Users/jinningwang/Documents/work/ams/ams/cases/matpower/case14.m"...
Detected raw file encoding: {'encoding': 'ascii', 'confidence': 1.0, 'language': ''}
CASE14    Power flow data for IEEE 14 bus test case.
Input file parsed in 0.0479 seconds.
System set up in 0.0009 seconds.


In [6]:
Cft, Bbus, Bf, Pbusinj, Pfinj = sp.mats.mBdc()

f=[ 0  0  1  1  1  2  3  3  3  4  5  5  5  6  6  8  8  9 11 12], t=[ 1  4  2  3  4  3  4  6  8  5 10 11 12  7  8  9 13 10 12 13]


In [ ]:
F_BUS = 0
T_BUS = 1
branch = ppci['branch']
nl=branch.shape[0]

f = np.real(branch[:, F_BUS]).astype(np.int64)                           ## list of "from" buses
t = np.real(branch[:, T_BUS]).astype(np.int64)                           ## list of "to" buses
i = np.r_[range(nl), range(nl)]                   ## double set of row indices

print(f"bus={ppci['bus'][:, 0]}")
print(f"f={f}\nt={t}\ni={i}")

In [ ]:
F_BUS = 0
T_BUS = 1
branch = ppc['branch']
nl=branch.shape[0]

f = np.real(branch[:, F_BUS]).astype(np.int64)                           ## list of "from" buses
t = np.real(branch[:, T_BUS]).astype(np.int64)                           ## list of "to" buses
i = np.r_[range(nl), range(nl)]                   ## double set of row indices

print(f"bus={ppc['bus'][:, 0]}")
print(f"f={f}\nt={t}\ni={i}")

In [ ]:
ppc['branch'][:, 0]

In [ ]:
ppc['branch'][:, 1]

In [ ]:
ppc['bus'][:, 0]

In [ ]:
Bbus.shape

In [ ]:
print(f"ppc: bus-{ppc['bus'].shape}, branch-{ppc['branch'].shape}, gen-{ppc['gen'].shape}")
print(f"ppc: bus-{ppci['bus'].shape}, branch-{ppci['branch'].shape}, gen-{ppci['gen'].shape}")

In [ ]:
# pd.DataFrame(ppci['branch']).round(1)

In [ ]:
# pd.DataFrame(ppc['branch'])

In [ ]:
self = sp.mats

nb = self.system.Bus.n
nl = self.system.Line.n

u = self.system.Line.u.v.astype(bool)       # in-service lines
b = self.system.Line.x.v[u]                 # series susceptance
tap = self.system.Line.tap.v[u] * rad2deg   # tap ratio
b = b / tap

f = self.system.Bus.idx2uid(self.system.Line.bus1.v)    # List of "from" buses
t = self.system.Bus.idx2uid(self.system.Line.bus2.v)    # List of "to" buses
f = np.array(f)[u]
t = np.array(t)[u]
i = np.concatenate((np.arange(nl), np.arange(nl)))      # Double set of row indices
v = np.concatenate((np.ones(nl), -np.ones(nl)))         # Values for the connection matrix

# Connection matrix
Cft = csr_matrix((v, (i, np.concatenate((f, t)))), shape=(nl, nb))

print(nl, nb)

In [ ]:
sp.Line.as_df().head(3)

In [ ]:
ppc['branch'][:, 0]

In [ ]:
ppc['bus'][:, 0]

In [ ]:
f

In [ ]:
sp.mats.mBdc()

In [ ]:
Bbus, Bf, ptdf = sp.mats.PTDF

In [ ]:
ppn = ppnet.case14()

pp.rundcpp(ppn)

_, ppci = _pd2ppc(ppn)

ppptdf = ppmakePTDF(ppci["baseMVA"], ppci["bus"], ppci["branch"],)


In [ ]:
ptdf.shape

In [ ]:
ppptdf.shape

In [ ]:
pp.rundcpp(ppc)

ptdf = ppmakePTDF(ppc["baseMVA"], ppc["bus"], ppc["branch"],
                  )

In [ ]:
PTDF = makePTDF(ppc["baseMVA"], ppc["bus"], ppc["branch"])

In [ ]:
ppc = sp.